In [57]:
import requests # Библиотека для отправки запросов
page_link = 'https://mangahub.ru'
response = requests.get(page_link + '/explore')
response

<Response [200]>

In [58]:
from bs4 import BeautifulSoup # библиотека для синтаксического разбора файлов HTML/XML, которая может преобразовать даже неправильную разметку в дерево синтаксического разбора

# распарсили страничку в дерево 
tree = BeautifulSoup(response.content, 'html.parser')

In [59]:
tree.html.head.title.text # Получим заголовок старницы каталога

'Каталог манги'

In [60]:
info = [] # Здесь с помощью циклов собираем в список словари с информацией о всъ книгах со станиц каталога
for i in range(1, 41):
    response = requests.get(page_link + '/explore?page=' + str(i))
    tree = BeautifulSoup(response.content, 'html.parser')
    books = tree.find_all('div', {'class': 'col-6 col-md-3 col-lg-4 comic-grid-col-xl mb-4 mb-md-3'})
    for book in books:
        title = book.find('a', {'class': 'comic-grid-name my-3'}).text.strip() # инфа по книге
        link = book.find('a', {'class': 'comic-grid-name my-3'})['href'] # ссылка
        genres = book.find('div', {'class': 'comic-grid-extra text-muted mt-1'}).find_all('a') # теги
        g = []
        for genre in genres:
            g.append(genre.text.strip().replace(',', '')) # объединяем теги в список
        book_info = {
             'title': title,
             'link': link,
             'genres': g
             }   # помещаем всю инфу по книге в словарь 
        dop = go_to_link(page_link + link) 
        book_info.update(dop) # объединяе два словаря с иформацией о комиксах со станиц каталога и с инфой внутри ссылки на конкретный комикс
        info.append(book_info) # Добавляем в список словарь с названием, ссылкой и списком жанров
        print(dop) # печатаем 

{'Всего просмотров': 8890, 'Кол-во в закладках': '187', 'Сказали спасибо': '384', 'В избранном': '71', 'Перевод': 'Продолжается', 'Дата обновления': '15 минут назад', 'Страна': 'Китай', 'Томов': '2, выпуск продолжается', 'Формат выпуска': 'В цвете', 'Автор': 'Mad Snail', 'Издательство': 'AC.QQ (Tencent)', 'Переводчик': 'КАНСЭЙ', 'Другие названия': 'Tales of Demons and Gods / 妖神记（全彩）', 'year': 2015, 'rating': 4.4}
{'Всего просмотров': 2770, 'Кол-во в закладках': '124', 'Сказали спасибо': '201', 'В избранном': '25', 'Перевод': 'Продолжается', 'Дата обновления': '17 минут назад', 'Страна': 'Корея', 'Томов': '2, выпуск продолжается', 'Формат выпуска': 'Веб', 'Сценаристы': 'Ю Ин, \n\n                        Сомттам', 'Художник': 'CMJ', 'Издательства': "Kakao\n\n\n\nToYou's Dream", 'Переводчики': 'Magic secret\n\n\n\nCrown of light', 'Другие названия': 'The lady want to rest / 쉬고 싶은 레이디 / The Lady Needs a Break / Lady Wants to Rest / 小姐想休息 / 想休息的小姐 / Леди хочет отдохнуть', 'year': 2021, 'rat

KeyboardInterrupt: 

In [ ]:
# Данная функция собирает инфу с переходом по ссылкам на страницу каждого комикса, собираем оттуда количество просмотров, год выпуска, рейтинг и прочую инфу из раздела Дополнительные данные
def go_to_link(link):
    dop = {}
    response = requests.get(link)
    tree = BeautifulSoup(response.content, 'html.parser')
    metrics = tree.find_all('div', {'class': 'detail-metric min-w-0'})
    for metric in metrics:
        title = metric.find('div', {'class': 'd-none d-md-block fs-2 fw-medium text-muted text-truncate mb-1'}).text.strip()  
        value = metric.find('span').text.strip()
        if value [-1] == 'К':
            value = int(float(value[:-1]) * 1000)
        elif value [-1] == 'M':
            value = int(float(value[:-1]) * 1000000)
        else:
            value = (value)
        dop[title] = value
        
    ditails = tree.find('div', {'class': 'detail-attrs'})
    ditails = tree.find_all('div', {'class': 'detail-attr'})
    for d in ditails:
        name = d.find('div', {'class': 'detail-attr-name'}).text.strip()  
        value = d.find('div', {'class': 'detail-attr-value'}).text.strip()  
        dop[name] = value
            
    top_info = tree.find('div', {'class': 'fs-2 text-muted fw-medium d-flex align-items-center'})
    top_info = top_info.find_all('span')
    year  = None
    for span in top_info: 
        if span.text.strip().isdigit():
            year = int(span.text.strip())
    dop['year'] = year
      
    rating = tree.find('span', {'class': 'fw-bold fs-2 fs-lg-5'}).text.strip() 
    dop['rating'] = float(rating)
            
    return dop

In [ ]:
import pandas as pd # имортируем библиотеку для создания таблиц по собранным данным 

df = pd.DataFrame(info)
display(df)

,title,link,genres,Всего просмотров,Кол-во в закладках,Сказали спасибо,В избранном,Перевод,Дата обновления,Страна,...,year,rating,Сценаристы,Художник,Переводчики,Автор,Издательства,Авторы,Сценарист,Художники
0,Дневник палаты по уголовным делам,/title/white_cat_legend,"[Исторический, История, Комедия, Мистика, Прик...",1130,52,251,12,Продолжается,23 минуты назад,Китай,...,2020.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Под дубом,/title/under_the_oak_tree,"[Романтика, Фэнтези, Драма, Психология]",61960,1310,1390,274,Продолжается,35 минут назад,Корея,...,2020.0,4.9,"Ким Су Джи, \n\n Namu",Пи,EDEN\n\n\n\nMELPOMENE TEAM,NaN,NaN,NaN,NaN,NaN
2,Симбиоз между кроликом и чёрной пантерой,/title/the_symbiotic_relationship_between_a_pa...,"[Сёдзё, Драма, Романтика, Фэнтези]",23150,822,1060,194,Продолжается,45 минут назад,Корея,...,2021.0,4.9,NaN,NaN,NaN,Садам,NaN,NaN,NaN,NaN
3,Рыжий лис - неудачник,/title/the_loser_fox_orange,"[Сёдзё, Фантастика, Комедия]",1940,105,258,28,Продолжается,46 минут назад,Китай,...,2019.0,4.9,NaN,NaN,SanSara\n\n\n\nEulavon,NaN,AC.QQ (Tencent)\n\n\n\nKuaikan Manhua,NaN,NaN,NaN
4,Замахнись сильнее,/title/big_windup,"[Спорт, Повседневность, Сэйнэн, Школа, Комедия...",719,10,21,7,Продолжается,4 часа назад,Япония,...,NaN,5.0,NaN,NaN,Улитка\n\n\n\nShutout Team\n\n\n\nk-kun no sora,Хигути Аса,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,Секретный Друг,/title/the_secret_friend,"[Эротика, Этти, Сэйнэн, Романтика, Комедия, Др...",4030,21,1,7,Продолжается,8 месяцев назад,Корея,...,2018.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1916,"Спасибо, доктор",/title/thank_you_doctor,"[Драма, Романтика, Психология]",1030,48,41,10,Продолжается,8 месяцев назад,Китай,...,2020.0,5.0,NaN,NaN,NaN,Cюэ Ян,NaN,NaN,NaN,NaN
1917,Тайная любовь и тайный брак,/title/secret_love_and_secret_mirrage,"[Сёдзё, Комедия, Романтика, Драма]",500,26,2,8,Продолжается,8 месяцев назад,Китай,...,2018.0,5.0,NaN,NaN,NaN,Би Гэ Сюн,NaN,NaN,NaN,NaN
1918,Академия бога демонов,/title/demon_god_academy,"[Героическое фэнтези, Школа, Романтика, Комеди...",85,3,0,0,Продолжается,8 месяцев назад,Китай,...,2020.0,0.0,NaN,NaN,NaN,Цзи Мань Вэньхуа,NaN,NaN,NaN,NaN


In [61]:
df.describe() # общий подсчет по числовым показателям

,year,rating
count,1762.000000,1920.000000
mean,2017.780931,3.898750
std,5.326556,1.883902
min,1968.000000,0.000000
25%,2017.000000,4.200000
50%,2019.000000,4.900000
75%,2021.000000,5.000000
max,2022.000000,5.000000


In [ ]:
# Здесь мы видим, что самым популярным автором является Юэ Вэнь Маньхуа
df = pd.DataFrame(info)
df['Автор'].value_counts()

Юэ Вэнь Маньхуа          15
Цзи Мань Вэньхуа          4
Шэньцзюй Дунмань          3
Акэ Вэньхуа               3
Юлу Вэньхуа               3
                         ..
Харуба Нэги               1
Magic Mountain Movies     1
Сакура Амю                1
harudayy                  1
Би Гэ Сюн                 1
Name: Автор, Length: 1124, dtype: int64

In [ ]:
# Здесь мы видим, что самой популярной страной издания комиксов является Япония
df = pd.DataFrame(info)
df = pd.DataFrame(info)
#df.info()
df['Страна'].value_counts()

Япония            711
Корея             708
Китай             338
США                37
Индонезия           7
Испания             3
Великобритания      3
Англия              2
Германия            2
Дания               1
Италия              1
Канада              1
Мексика             1
Тайвань             1
Name: Страна, dtype: int64

In [ ]:
# Здесь мы видим, в основном стус перевода продолжается, из этого можно сдлать вывод, что год выпуска манги относительно недавний, проверим это ниже
df = pd.DataFrame(info)
#df.info()
df['Перевод'].value_counts()

Продолжается    1673
Завершен         215
Заброшен          25
Заморожен          7
Name: Перевод, dtype: int64

In [ ]:
# Здесь мое предположение подтвердилось, больше всего комиксов было издано в 2021 году
df = pd.DataFrame(info)
df['year'].value_counts()

2021.0    434
2020.0    295
2019.0    216
2018.0    174
2022.0    128
2017.0    123
2016.0     84
2015.0     62
2014.0     50
2013.0     31
2012.0     24
2011.0     23
2010.0     17
2009.0     14
2006.0      9
2008.0      9
2007.0      8
1998.0      5
2000.0      5
2005.0      5
2004.0      5
1997.0      4
1999.0      4
2002.0      4
1994.0      4
2001.0      4
1993.0      4
2003.0      3
1990.0      3
1988.0      2
1989.0      2
1970.0      1
1996.0      1
1992.0      1
1968.0      1
1971.0      1
1991.0      1
1977.0      1
Name: year, dtype: int64

In [ ]:
# Здесь видим, что рейтинг у комиксов в основном хороший и комиксов с низким рейтингом буквально единицы
df = pd.DataFrame(info)
#df.info()
df['rating'].value_counts()

5.0    861
0.0    347
4.9    139
4.8    130
4.7     98
4.5     79
4.0     62
4.6     59
4.3     44
4.4     20
3.0     20
4.2     18
1.0     10
3.8      8
3.7      8
3.5      5
4.1      4
3.3      4
2.0      2
2.5      1
3.6      1
Name: rating, dtype: int64

In [ ]:
# Здесь видим, что самое популярное издательство это "Kakao"
df = pd.DataFrame(info)
df['Издательство'].value_counts()

Kakao              225
Naver              158
AC.QQ (Tencent)    126
Kodansha           114
Webtoon             71
                  ... 
Libre Shuppan        1
U17                  1
Compass              1
BookLive             1
Shinchousha          1
Name: Издательство, Length: 84, dtype: int64

In [ ]:
# Таким образом можем вывести самый популярный набор жанров
df = pd.DataFrame(info)
df['genres'].value_counts()

[Школа, Сёдзё, Романтика]                                                         21
[Романтика, Сёдзё, Фэнтези]                                                       20
[История, Сёдзё, Драма, Романтика, Фэнтези]                                       20
[Сёдзё, Драма, Романтика, Фэнтези]                                                17
[Романтика, Сёдзё, Драма]                                                         15
                                                                                  ..
[Самурайский боевик, Ужасы, Трагедия, Сэйнэн, Сверхъестественное, Приключения]     1
[Фэнтези, Боевые искусства, Сёнэн]                                                 1
[Комедия, Сёдзё, Фэнтези]                                                          1
[Приключения]                                                                      1
[Героическое фэнтези, Школа, Романтика, Комедия, Фэнтези, Приключения]             1
Name: genres, Length: 1266, dtype: int64

In [62]:
df.sort_values('В избранном', ascending=False) # топ по избранному

,title,link,genres,Всего просмотров,Кол-во в закладках,Сказали спасибо,В избранном,Перевод,Дата обновления,Страна,...,year,rating,Сценаристы,Художник,Переводчики,Автор,Издательства,Авторы,Сценарист,Художники
1014,Свинарник,/title/pigpen,"[Ужасы, Трагедия, Сверхъестественное, Сэйнэн, ...",7120,448,249,99,Завершен,1 месяц назад,Корея,...,2019.0,4.4,NaN,NaN,NiVeGroup\n\n\n\nKlown Team,NaN,NaN,"Ким Кханби (Карнби), \n\n ...",NaN,NaN
415,"Для того, чтобы помочь тебе понять",/title/what_it_means_to_be_you,"[Сёдзё, История, Драма]",11690,394,872,99,Продолжается,9 дней назад,Корея,...,2021.0,4.9,NaN,NaN,NaN,О Сён,NaN,NaN,NaN,NaN
167,"Я мачеха, но моя дочь слишком милая",/title/ia_macheha_no_moia_doch_slishkom_milaia,"[Сёдзё, Романтика]",9190,483,992,98,Продолжается,2 дня назад,Корея,...,2021.0,4.9,NaN,NaN,NaN,Мо Гурэ́н,NaN,NaN,NaN,NaN
221,Когда ты становишься мишенью для хулигана,/title/when_youre_targeted_by_the_bully,"[Сёдзё, Школа, Комедия, Романтика, Драма]",9270,408,459,97,Продолжается,4 дня назад,Корея,...,2020.0,4.7,NaN,NaN,NaN,Чхедя,NaN,NaN,NaN,NaN
472,Айдол с Небес,/title/idol_is_a_god,"[Фэнтези, Сёдзё, Приключения]",6690,361,1060,97,Завершен,11 дней назад,Китай,...,2019.0,4.6,NaN,NaN,NaN,Будуй Хо Го,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1635,Тодайские мстители,/title/toudai_revengers,"[Школа, Комедия]",376,5,0,0,Продолжается,6 месяцев назад,Япония,...,2021.0,0.0,NaN,NaN,NaN,FUNATSU Shinpei,NaN,NaN,NaN,NaN
1546,Могучие Рейнджеры: Подростки Мутанты Ниндзя Че...,/title/mighty_morphin_power_rangers_teenage_mu...,"[Научная фантастика, Фантастика, Боевик, Боевы...",31,0,0,0,Продолжается,5 месяцев назад,NaN,...,2019.0,0.0,NaN,Симоне де Мио,Supercomics\n\n\n\nExcalibur-Team,NaN,NaN,NaN,Райан Пэррот,NaN
1632,Звёздные войны: Эпоха Сопротивления,/title/star_wars_age_of_resistance,"[Научная фантастика, Героическое фэнтези, Драм...",21,1,0,0,Завершен,6 месяцев назад,США,...,2019.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Том Тейлор,"Леонард Кирк, \n\n Rosa..."
1516,Неопознанный Принца Объект,/title/mikakunin_prince_buttai_u_p_o,"[Сэйнэн, Романтика, Фэнтези, Боевик, Приключения]",38,0,0,0,Продолжается,5 месяцев назад,Япония,...,1990.0,0.0,NaN,NaN,NaN,Tanaka Masashi,NaN,NaN,NaN,NaN


In [79]:
df.sort_values('rating', ascending=False) # топ по рейтингу

,title,link,genres,Всего просмотров,Кол-во в закладках,Сказали спасибо,В избранном,Перевод,Дата обновления,Страна,...,year,rating,Сценаристы,Художник,Переводчики,Автор,Издательства,Авторы,Сценарист,Художники
0,Дневник палаты по уголовным делам,/title/white_cat_legend,"[Исторический, История, Комедия, Мистика, Прик...",1130,52,251,12,Продолжается,23 минуты назад,Китай,...,2020.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
959,Дорога от первого уровня к статусу сильнейшего...,/title/summon_warriors_starting_from_level_1,[Фэнтези],281,7,3,1,Продолжается,1 месяц назад,Япония,...,2022.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1018,Злодейка Каруна стала младше,/title/the_villainess_karuna_has_become_a_child,[История],3630,115,49,19,Продолжается,1 месяц назад,Корея,...,2021.0,5.0,NaN,Deokbae,NaN,NaN,NaN,NaN,Moon Kyung,NaN
1008,Бессмертный камикадзе,/title/bessmertnyi_kamikadze,"[История, Сэйнэн, Боевик, Драма, Приключения]",278,9,0,2,Продолжается,1 месяц назад,Япония,...,2018.0,5.0,NaN,NaN,NaN,NaN,NaN,"Адзума Наоки, \n\n Кока...",NaN,NaN
1006,Фрагменты памяти,/title/memory_fragments,"[Драма, Романтика, Психология]",166,8,5,3,Завершен,1 месяц назад,NaN,...,2018.0,5.0,NaN,NaN,NaN,bangjoy2021,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062,Блаттода,/title/blattodea,"[Этти, Ужасы, Трагедия, Драма, Боевик, Психоло...",170,2,0,0,Продолжается,2 месяца назад,Япония,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1067,Соара и Дом монстров,/title/soara_to_mamono_no_ie,"[Комедия, Фэнтези, Драма, Приключения]",27,0,0,0,Продолжается,2 месяца назад,Япония,...,2021.0,0.0,NaN,NaN,NaN,Yamaji Hidenori,NaN,NaN,NaN,NaN
363,История об очень обычной парочке,/title/gokugoku_futsu_no_fufu_no_hanashi,"[Повседневность, Романтика]",441,23,1,7,Продолжается,8 дней назад,Япония,...,2021.0,0.0,NaN,NaN,Pigman\n\n\n\nLunina,Токива,NaN,NaN,NaN,NaN
1586,В следующем месяце я стану героем,/title/raigetsu_hiro_no_naru_boku_no_hanashi,"[Повседневность, Романтика]",5,0,0,0,Продолжается,6 месяцев назад,Япония,...,2021.0,0.0,NaN,NaN,NaN,Perico,NaN,NaN,NaN,NaN
